# 2A.algo - Plus proches voisins en grande dimension

La méthodes des [plus proches voisins](https://fr.wikipedia.org/wiki/Recherche_des_plus_proches_voisins) est un algorithme assez simple. Que se passe-t-il quand la dimension de l'espace des features augmente ? Comment y remédier ? Le profiling [memory_profiler](https://pypi.python.org/pypi/memory_profiler) ou [cprofile]() sont des outils utiles pour savoir où le temps est perdu. 

In [ ]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Q1 : k-nn : mesurer la performance

In [ ]:
from sklearn.datasets import make_classification
datax, datay = make_classification(10000, n_features=10, n_classes=3, 
                                   n_clusters_per_class=2, n_informative=8)
datax[:3]

array([[ 0.51351564, -1.29129013,  2.5460118 , -2.42104426, -0.81367567,
        -1.47768628,  0.60560016, -0.21796964,  1.93135557, -0.64311937],
       [ 1.54541078,  3.41301354,  6.13334634,  1.26794708, -1.11676539,
         1.24866106, -3.63118124,  0.0403887 , -0.98677694, -1.17217912],
       [ 0.46325456,  0.89963871,  2.444065  , -1.50433353, -0.61699517,
        -0.09272635, -0.49436294, -0.53280471,  2.82896846,  2.06995585]])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(5, algorithm="brute")
model.fit(datax, datay)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [ ]:
model.predict(datax)

array([1, 0, 2, ..., 1, 2, 1])

In [ ]:
%timeit model.predict(datax)

2.33 s ± 50.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
res = %prun -q -r model.predict(datax)

In [ ]:
res.strip_dirs().sort_stats("cumtime").print_stats()

         215 function calls in 2.328 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.328    2.328 {built-in method builtins.exec}
        1    0.000    0.000    2.328    2.328 <string>:1(<module>)
        1    0.065    0.065    2.328    2.328 classification.py:129(predict)
        1    0.072    0.072    2.261    2.261 base.py:273(kneighbors)
        1    0.000    0.000    1.112    1.112 pairwise.py:1141(pairwise_distances)
        1    0.000    0.000    1.112    1.112 pairwise.py:1079(_parallel_pairwise)
        1    0.329    0.329    1.112    1.112 pairwise.py:163(euclidean_distances)
        2    0.000    0.000    1.076    0.538 fromnumeric.py:55(_wrapfunc)
        1    0.000    0.000    1.075    1.075 fromnumeric.py:643(argpartition)
        1    1.075    1.075    1.075    1.075 {method 'argpartition' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.783    0.783 extmath.py:1

Etudier l'évolution du temps de prédiction en fonction du nombre d'observations, de la dimension, du nombre de classes ? Qu'en déduisez-vous ? Le code sur GitHub :

* [predict](https://github.com/scikit-learn/scikit-learn/blob/ef5cb84a/sklearn/neighbors/classification.py#L129)
* [kneighbors](https://github.com/scikit-learn/scikit-learn/blob/ef5cb84a805efbe4bb06516670a9b8c690992bd7/sklearn/neighbors/base.py#L273)
* [pairwise_distance](https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/metrics/pairwise.py#L1141)

## Q2 : k-nn avec sparse features

On recommence cette mesure de temps mais en créant des jeux de données [sparses](https://fr.wikipedia.org/wiki/Matrice_creuse).

## Q3 : Imaginez une façon d'aller plus vite ?

Aller plus vite veut parfois dire perdre un peu en performance et dans notre cas, on veut accélérer la prédiction.